In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"//data//"+p, data_list)) #
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240731_0817.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240731_0217.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240731_0017.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240716_2142.raw']


In [ ]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)
import matplotlib.pyplot as plt

interval = 1_00 # Set your monitoring chunck size
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 50_000 # End the loop early
    
for file in file_path:
    fReader = rawFileReader.fileReader(file) # load in the classs object
    processedEvents = 0 # Initialisation
    noiseStart = 0
    firstNoise = False
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    originTime = initial_event_chunk[0].tdcEvents[0].time
    mets = [[] for tdc in range(5)]
    mets_off = [[] for tdc in range(5)]
    file_name = file.split("//")[-1][:-4]
    tdc_mets = [[] for tdc in range(5)]
    Tot_TDC_info = [[] for tdc in range(5)]
    chunks = []
    with tqdm(total=max_process_event_chunk, desc=f"Processing Events {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            try:
                event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True) # get the aligned events
            except Exception as e:
                print(e)
                break
            [tdc_mets[i].append(tdc_met[i]) for i in range(5) if tdc_met[i] != 0]
            [Tot_TDC_info[i].extend(TDC_info[i]) for i in range(5) if TDC_info[i]]
            for idx, (i, j) in enumerate(order):
                x, y, l, m = ATools.find_tdc_alignment_metric(i, j) # determining which RPCs to use for aignment metric
                alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents) # determine the alignment metric
                #alignMet_off = ATools.calcAvgAlign(event_chunk, 15, x, y, l, m, i, j, processedEvents) # determine the alignment metric
                
                #mets_off[idx].append(alignMet_off) # write to memory
                mets[idx].append(alignMet) # write to memory
                if alignMet > 40 and not firstNoise:
                    for tdc in range(5):
                        if event_chunk[0].tdcEvents[tdc].time != 0:
                            noiseTime = event_chunk[0].tdcEvents[tdc].time
                            break
                    if noiseTime == 0:
                        print("No hits in the event???")
                    startTime =  noiseTime - originTime
                    print(startTime)
                    noiseStart = processedEvents*interval
                    firstNoise = True
            pbar.update(1)
    
    #plot_alignment(mets_off, order, max_process_event_chunk, file_name+"_off")
    #plot_alignment(mets, order, max_process_event_chunk, file_name+"fuck")
    #plot_bvg(tdc_mets, max_process_event_chunk, file_name+"fuck")
    #plot_bvg(tdc_mets, max_process_event_chunk, file_name)
    print(noiseStart)
    #TTools.plot_tdc_error_times_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], output_pdf=f'output//transition//Time_Hit_{file_name}.pdf')
    TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel_perEv_{file_name}.pdf')
